### Визначення найбільш відвідуваних квадратів для кожного із абонентів
- розбити місто на квадрати;
- визначити для кожного користувача, які квадрати 

__To Do:__ Сортувати по кількості подій від найбільшої до найменшої для кожного абонента; виводити перших 2-3 квадрати

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("BD_LAB_EXAMPLE_SAMPLE.csv",encoding="cp1251")

/home/ogilvy/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,7,8,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Convert column `envent_start_date` (`str`) into  `datetime`
df.event_start_date = pd.to_datetime(df.event_start_date, format='%d%b%y:%H:%M:%S')

# Convert column 'cost' (`str`) into `float`
df['cost'] = pd.to_numeric(df['cost'],errors='coerce')

In [5]:
# Delete rows without latitude -- where LAT = '.'
# Write to new dataframe df_with_locat
df_with_locat = df[df['LAT'] != '.']

# Convert latitude and longitude from `str` into `float`
df_with_locat['LAT'] = pd.to_numeric(df_with_locat['LAT'], errors = 'coerce')
df_with_locat['LON'] = pd.to_numeric(df_with_locat['LON'], errors = 'coerce')

/home/ogilvy/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ogilvy/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Кількість людей в радіусі від центру міста (Львова)

In [7]:
# Lviv center coordinates
center_lat, center_lon = 49.84, 24.03  # Lviv
#center_lat, center_lon = 48.28, 25.94  # Chernivtsi


## calculate length in km of 1 degree for latitude and longitude
a = 6378.137; b = 6356.752 # semiaxis of Earth ellipsoid in km
e2 = (a**2 - b**2) / a**2 # eccentricity of Earth

# Convert degree to radians
def deg_to_rad(phi):
    return np.pi * phi / 180

# one degree of latitude and longitude in km
deg_lat_in_km = np.pi / 180 * a * (1-e2) /\
            (1 - e2*np.sin(deg_to_rad(center_lat)))**1.5
deg_lon_in_km = np.pi / 180 * a * np.cos(deg_to_rad(center_lat)) /\
             np.sqrt(1 - e2*np.sin(deg_to_rad(center_lat))**2 )

print("1° of latitude = {0:5.2f} km".format(deg_lat_in_km)) 
print("1° of longitude = {0:5.2f} km".format(deg_lon_in_km))

1° of latitude = 111.43 km
1° of longitude = 71.93 km


In [8]:
#Convert distance in km to degrees
def km_to_lat(x):
    return x / deg_lat_in_km

def km_to_lon(x):
    return x / deg_lon_in_km

In [9]:
# Radius of Lviv 
city_radius = 10 # km

# check if point is inside circle with radius `city_radius` and center in (center_lat, center_lon)
def isInCity_radius(lat, lon, center_lat, center_lon, city_radius):
    """True if (lat,lon) is in City, False otherwise"""
    return ((lat - center_lat)**2 / km_to_lat(city_radius)**2 +
            (lon - center_lon)**2 / km_to_lon(city_radius)**2) < 1


df_Lviv = df_with_locat[isInCity_radius(df_with_locat['LAT'],df_with_locat['LON'], 
                                 center_lat, center_lon, city_radius)]


In [10]:

len(df_Lviv)

45077

In [11]:
# add column with number of square
# center of the city has number (0, 0)


# square size
square_side_in_km = .5 # km
dlat = km_to_lat(square_side_in_km) # square side in latitude degrees
dlon = km_to_lon(square_side_in_km) # square side in longitude degrees

df_Lviv['square'] = list(zip(np.rint((df_Lviv['LAT'] - center_lat) / dlat ), 
                             np.rint((df_Lviv['LON'] - center_lon) / dlon)))

/home/ogilvy/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
df_Lviv['square'].head()

14420     (-10.0, 5.0)
14421    (-15.0, 14.0)
14427     (-10.0, 5.0)
14430     (-5.0, 16.0)
14434     (-10.0, 5.0)
Name: square, dtype: object

In [13]:
# determine number of calls for every `hash_number_A` from different squares
square_visits = df_Lviv.groupby(['hash_number_A', 'square']).size()


In [14]:
square_visits.head(40)

hash_number_A  square       
171195         (-15.0, -3.0)      3
               (-14.0, -1.0)      1
172162         (-13.0, 10.0)      1
               (-1.0, -5.0)       2
               (4.0, -7.0)        1
172183         (-13.0, 6.0)       1
               (-7.0, -5.0)       1
               (-1.0, -5.0)       2
               (0.0, -4.0)        1
               (0.0, 1.0)         3
173437         (-15.0, -3.0)      1
               (-14.0, 4.0)       1
               (-5.0, -15.0)      1
               (-3.0, -16.0)      3
               (6.0, -14.0)       1
               (7.0, -15.0)       2
               (12.0, 14.0)       1
173781         (-15.0, -0.0)      6
               (-14.0, -1.0)      2
               (-12.0, -2.0)      1
               (-10.0, -1.0)      2
               (-9.0, -6.0)       2
               (-8.0, -4.0)       3
               (-7.0, -5.0)      19
               (-7.0, -4.0)     159
               (-7.0, -2.0)       2
               (-6.0, -8.0)       5

In [15]:
df_reset = square_visits.reset_index()
df_reset.head()

,hash_number_A,square,0
0,171195,"(-15.0, -3.0)",3
1,171195,"(-14.0, -1.0)",1
2,172162,"(-13.0, 10.0)",1
3,172162,"(-1.0, -5.0)",2
4,172162,"(4.0, -7.0)",1


In [16]:
df_reset.columns = ['hash_number_A', 'square', 'count']

In [23]:
ser = pd.Series(['a', 'b', 'c', 'ab', 'aa'])
ser

0     a
1     b
2     c
3    ab
4    aa
dtype: object

In [34]:
ser.sort_values()

0     a
4    aa
3    ab
1     b
2     c
dtype: object

In [25]:
ser

0     a
4    aa
3    ab
1     b
2     c
dtype: object

In [33]:
squares2 = df_reset.groupby(['hash_number_A', 'square'])['count'].apply(
    lambda x: sorted_values(x))
squares2.head()

NameError: name 'sorted_values' is not defined

In [30]:
squares2.head(10)

hash_number_A  square       
171195         (-15.0, -3.0)    None
               (-14.0, -1.0)    None
172162         (-13.0, 10.0)    None
               (-1.0, -5.0)     None
               (4.0, -7.0)      None
172183         (-13.0, 6.0)     None
               (-7.0, -5.0)     None
               (-1.0, -5.0)     None
               (0.0, -4.0)      None
               (0.0, 1.0)       None
Name: count, dtype: object

In [28]:
type(squares2)

pandas.core.series.Series

In [32]:
square_visits.head()

hash_number_A  square       
171195         (-15.0, -3.0)    3
               (-14.0, -1.0)    1
172162         (-13.0, 10.0)    1
               (-1.0, -5.0)     2
               (4.0, -7.0)      1
dtype: int64